# Maxwell chain: relaxation function

***

<center><i>Petr Havlásek (c) 2023, petr.havlasek@cvut.cz</i></center>

***

In [1]:
online = False

if (online):
    import micropip
    await micropip.install('ipywidgets')

import math
import numpy as np    
    
import matplotlib.pyplot as plt
    
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

from IPython.display import display


**Notation:**

>`E` ... spring stiffness [MPa]<br>

>`eta` ... dashpot viscosity [MPa day]<br>

>`tau` ... relaxation time [day] (=`eta/E`)<br>

>`t` ... time of interest [day]<br>

>`tt` ... time of loading [day]<br>

## Maxwell unit class

In [2]:
class Maxwell:
    def __init__(self, E, tau, activity = False):
        self.E = E
        self.tau = tau
        self.activity = activity

    # computes relaxation function

    def R_func (self, t, tt): 
        if t >= tt:
            dt = t-tt
    
            if self.tau > 0.:
                return self.E * math.exp(-(dt)/ self.tau) 
            else:
                return self.E
                
        else:
            return 0.


## Parameters definition

In [3]:
# Reference value of spring stiffness [GPa]
E_ref = 10.
# reference value of retardation time [day]
tau_ref = 1.

# number of Maxwell units (one of which is a spring)
N = 4

# ranges for parameters of the indiviudal Maxwell units specifies powers of 10
log_E_range = 1
log_tau_range = 2

# number of time points
t_div = 100

# list which stores Maxwell units, i.e. Maxwell chain
Maxwells = []

for i in range(N):
    
    E_i = E_ref

    if i == 0:
        tau_i = 0.
    else:
        tau_i = tau_ref * 10**(i)
        
    Maxwells.append( Maxwell(E_i, tau_i) )

# first full Maxwell unit is the only active unit
Maxwells[1].activity = True

# predefined settings
log_scale = False
draw_sum = False
draw_data = False
    

In [4]:
## Experimental data (Ross Dam concrete)

In [5]:
# t
data_t_28 = [0.041, 1.0, 2.0, 3.0, 4.0, 8.0, 9.0, 10.0, 12.1, 13.2, 15.1, 16.7, 19.9, 23.1, 24.9, 28.3, 29.7, 30.1, 39.1, 42.7, 46.6, 50.9, 59.1, 65.3, 71.3, 75.0, 79.8, 87.1, 88.2, 101.3, 113.3, 122.2, 138.5, 143.8, 155.0, 175.6, 180.1, 191.7, 201.5]
# MPa
data_R_28 = np.array([2.8062E+04, 2.5133E+04, 2.4801E+04, 2.3640E+04, 2.2838E+04, 2.2141E+04, 2.1577E+04, 2.0807E+04, 2.0205E+04, 2.0169E+04, 2.0502E+04, 1.9447E+04, 1.9211E+04, 1.8771E+04, 1.8775E+04, 1.8781E+04, 1.8131E+04, 1.7765E+04, 1.7615E+04, 1.6967E+04, 1.5911E+04, 1.8119E+04, 1.7678E+04, 1.6990E+04, 1.6709E+04, 1.5610E+04, 1.6429E+04, 1.6964E+04, 1.6353E+04, 1.5748E+04, 1.5632E+04, 1.5310E+04, 1.6214E+04, 1.4013E+04, 1.5934E+04, 1.6716E+04, 1.4188E+04, 1.4028E+04, 1.4276E+04])
# GPa
data_R_28 *= 1.e-3
# t
data_t_90 = [0.042, 1.0, 2.0, 3.1, 5.1, 6.5, 7.8, 9.9, 11.9, 14.2, 16.9, 20.4, 24.0, 30.8, 36.7, 40.1, 52.2, 61.4, 66.2, 69.6, 82.9, 98.8, 122.2, 140.2, 143.8, 171.3, 201.5]
# MPa
data_R_90 = np.array([3.3611E+04, 3.0314E+04, 2.9820E+04, 2.8904E+04, 2.8237E+04, 2.8372E+04, 2.7607E+04, 2.7089E+04, 2.7507E+04, 2.5517E+04, 2.4874E+04, 2.4965E+04, 2.4158E+04, 2.3314E+04, 2.2508E+04, 2.3288E+04, 2.3220E+04, 2.2168E+04, 2.2580E+04, 2.2419E+04, 2.1531E+04, 2.2438E+04, 2.0736E+04, 2.0376E+04, 1.9112E+04, 1.9081E+04, 1.8763E+04])
# GPa
data_R_90 *= 1.e-3
    

In [6]:
## Plotting and figure updating functions

In [10]:

def update_plot():
    with output:
        output.clear_output(wait = True)
        plot_Maxwell_chain_relaxation()
        
        display(fig)
    
def plot_Maxwell_chain_relaxation():       
        
    if (log_scale):
        times = np.logspace( round(math.log10(tau_ref)) - log_tau_range , round(math.log10(tau_ref)) + log_tau_range + 2, num = t_div )
    else:
        times = np.linspace(0., tau_ref * 10**(log_tau_range+1), num = t_div )

    ax.clear()


    if (draw_data):
        ax.scatter(data_t_28, data_R_28, lw=1., color="blue", label=r"$R, t'=28$ d")
        ax.scatter(data_t_90, data_R_90, lw=1., color="black", label=r"$R, t'=90$ d")

    tt = 0.
    
    R_tot = np.zeros(t_div)
    
    # loop over Maxwell units
    for max in Maxwells:

        # each Maxwell unit needs to be active to contribute to the relaxation
        if (max.activity):
            R_max = []
    
            for i in range(t_div):
                R_t_tt = max.R_func (times[i], tt) 
                R_max.append(R_t_tt)
                R_tot[i] += R_t_tt

            if ( Maxwells.index(max) == 0 ):
                label = "Spring"
            else:
                label = r'Maxwell$_{' + str( Maxwells.index(max) ) + "}$"  
                
            if ( Maxwells.index(max) == 0 ):
                color = "blue"
            elif ( Maxwells.index(max) == 1 ):
                color = "red"
            elif ( Maxwells.index(max) == 2 ):                
                color = "magenta"
            else:
                color = "cyan"
            
            ax.plot(times, R_max, lw=2., color=color, label=label)

            # plot vertical line corresponding to characteristic time
            if ( Maxwells.index(max) > 0 ):
                ax.axvline(x = max.tau, lw=2., color=color, linestyle = "--")
                

    # draws the relaxation function of the entire chain
    if (draw_sum):
        ax.plot(times, R_tot, color="black", lw = 3., label=r"$R_{tot}$")

    if (log_scale):
        ax.set_xscale('log')
        ax.set_xlim([ tau_ref * 10**(-log_tau_range), tau_ref * 10**(log_tau_range + 1)])
    else:
        ax.set_xlim([0., tau_ref * 10**(log_tau_range )])
    
    ax.legend()
    ax.set_xlabel('Duration of loading, $t-t\'$ [day]', fontsize=14)
    ax.set_ylabel('Relaxation, $R$ [GPa]', fontsize=14)

    ax.grid(True)


## GUI

In [11]:
step = 1./20.

## sliders
# degenerated Maxwell unit (spring)

i = 0

E = Maxwells[i].E

log_E_min = math.floor(math.log10(E)) - log_E_range
log_E_max = math.ceil(math.log10(E)) + log_E_range

E_0_slide = widgets.FloatLogSlider(min=log_E_min, max=log_E_max, value=E, step=step, description='E0 [GPa]', continuous_update=False, orientation='vertical')
Max_0_activity = widgets.Checkbox(value=Maxwells[i].activity, description='active')


###
# full Maxwell units
###

i = 1

E = Maxwells[i].E
tau = Maxwells[i].tau

log_E_min = math.floor(math.log10(E)) - log_E_range
log_E_max = math.ceil(math.log10(E)) + log_E_range
log_tau_min = math.floor(math.log10(tau)) - log_tau_range
log_tau_max = math.ceil(math.log10(tau)) + log_tau_range

E_1_slide = widgets.FloatLogSlider(min=log_E_min, max=log_E_max, value=E, step=step, description='E1 [GPa]', continuous_update=False, orientation='vertical')
tau_1_slide = widgets.FloatLogSlider(min=log_tau_min, max=log_tau_max, value=tau, step=step, description='tau1 [day]', continuous_update=False, orientation='vertical')
Max_1_activity = widgets.Checkbox(value=Maxwells[i].activity, description='active')

###

i = 2

E = Maxwells[i].E 
tau = Maxwells[i].tau

log_E_min = math.floor(math.log10(E)) - log_E_range
log_E_max = math.ceil(math.log10(E)) + log_E_range
log_tau_min = math.floor(math.log10(tau)) - log_tau_range
log_tau_max = math.ceil(math.log10(tau)) + log_tau_range

E_2_slide = widgets.FloatLogSlider(min=log_E_min, max=log_E_max, value=E, step=step, description='E2 [GPa]', continuous_update=False, orientation='vertical')
tau_2_slide = widgets.FloatLogSlider(min=log_tau_min, max=log_tau_max, value=tau, step=step, description='tau2 [day]', continuous_update=False, orientation='vertical')
Max_2_activity = widgets.Checkbox(value=Maxwells[i].activity, description='active')

###

i = 3

E = Maxwells[i].E
tau = Maxwells[i].tau

log_E_min = math.floor(math.log10(E)) - log_E_range
log_E_max = math.ceil(math.log10(E)) + log_E_range
log_tau_min = math.floor(math.log10(tau)) - log_tau_range
log_tau_max = math.ceil(math.log10(tau)) + log_tau_range

E_3_slide = widgets.FloatLogSlider(min=log_E_min, max=log_E_max, value=E, step=step, description='E3 [GPa]', continuous_update=False, orientation='vertical')
tau_3_slide = widgets.FloatLogSlider(min=log_tau_min, max=log_tau_max, value=tau, step=step, description='tau3 [day]', continuous_update=False, orientation='vertical')
Max_3_activity = widgets.Checkbox(value=Maxwells[i].activity, description='active')

###

log_scale_checkbox = widgets.Checkbox(value=log_scale, description='log-scale')
draw_sum_checkbox = widgets.Checkbox(value=draw_sum, description='draw sum')
draw_data_checkbox = widgets.Checkbox(value=draw_data, description='Ross Dam concrete (example)')


def on_E_0_change(change):
    Maxwells[0].E = change['new']
    update_plot()

def on_activity_0_change(change):
    Maxwells[0].activity = change['new']
    update_plot()

def on_E_1_change(change):
    Maxwells[1].E = change['new']
    update_plot()

def on_tau_1_change(change):
    Maxwells[1].tau = change['new']
    update_plot()    

def on_activity_1_change(change):
    Maxwells[1].activity = change['new']
    update_plot()    

def on_E_2_change(change):
    Maxwells[2].E = change['new']
    update_plot()
         

def on_tau_2_change(change):
    Maxwells[2].tau = change['new']
    update_plot()   

def on_activity_2_change(change):
    Maxwells[2].activity = change['new']
    update_plot()    
    
def on_E_3_change(change):
    Maxwells[3].E = change['new']
    update_plot()
         
def on_tau_3_change(change):
    Maxwells[3].tau = change['new']
    update_plot()   

def on_activity_3_change(change):
    Maxwells[3].activity = change['new']
    update_plot()    
    

def on_logscale_change(change):

    global log_scale
    log_scale = change['new']
    update_plot()    

def on_draw_sum_change(change):

    global draw_sum
    draw_sum = change['new']
    update_plot()    

def on_draw_data_change(change):

    global draw_data
    draw_data = change['new']
    update_plot()    



Max_0_activity.observe(on_activity_0_change, names = 'value')
E_0_slide.observe(on_E_0_change, names = 'value')

Max_1_activity.observe(on_activity_1_change, names = 'value')
E_1_slide.observe(on_E_1_change, names = 'value')
tau_1_slide.observe(on_tau_1_change, names = 'value')

Max_2_activity.observe(on_activity_2_change, names = 'value')
E_2_slide.observe(on_E_2_change, names = 'value')
tau_2_slide.observe(on_tau_2_change, names = 'value')

Max_3_activity.observe(on_activity_3_change, names = 'value')
E_3_slide.observe(on_E_3_change, names = 'value')
tau_3_slide.observe(on_tau_3_change, names = 'value')


log_scale_checkbox.observe(on_logscale_change, names = 'value')
draw_sum_checkbox.observe(on_draw_sum_change, names = 'value')
draw_data_checkbox.observe(on_draw_data_change, names = 'value')


par_0 = widgets.VBox([Max_0_activity, E_0_slide])

par_1 = widgets.VBox([Max_1_activity, widgets.HBox([E_1_slide, tau_1_slide])])

par_2 = widgets.VBox([Max_2_activity, widgets.HBox([E_2_slide, tau_2_slide])])

par_3 = widgets.VBox([Max_3_activity, widgets.HBox([E_3_slide, tau_3_slide])])


pars = widgets.HBox([par_0, par_1, par_2, par_3])



## Updating 

In [12]:

output = widgets.Output()

fig, ax = plt.subplots(1, 1, figsize=(10,5))

plt.rcParams.update({'font.size': 14})
plt.close(fig)

update_plot()

display(widgets.HBox([log_scale_checkbox, draw_sum_checkbox, draw_data_checkbox]), output, pars)


Output()